In [72]:
import requests
import lxml
import pandas as pd
from bs4 import BeautifulSoup # Html search tool
import re # Regular expressions

In [ ]:
response = requests.get("https://www.zillow.com/homes/Raleigh,-NC_rb/")
print(response.text)

In [55]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
   city = 'raleigh/' #*****change this city to what you want*****
   url = 'https://www.zillow.com/homes/for_sale/'+city    
   r = s.get(url, headers=req_headers)

In [26]:
#add contents of urls to soup variable from each url
soup = BeautifulSoup(r.content, "html.parser")

<built-in method capitalize of str object at 0x7f8bd2db5570>


In [ ]:
soup.find_all("a")

In [ ]:
# From intial queried page (e.g., homes in Raleigh) find all links to homedetails page,
# which will have more detailed information on the home.

# This will be the main base where the scraper will come back to after scraping each
# home details page. 

# Example: queried page -> homedetails page -> scrape data -> queried page 
# Loop until all homes are scraped, then proceed to next page.
soup.find_all(href=re.compile("homedetails"))
# soup.select('a[href^="https://www.zillow.com/homedetails/"]')


In [82]:
def find_all_keyword_units(keyword):
    """Returns a list of keyword matches"""
    keyword_match_list = soup.find_all(href=re.compile(str(keyword)))
    return keyword_match_list


In [ ]:
# Simply testing function, can remove
print(find_all_keyword_units(keyword = "homedetails"))

page 1:

https://www.zillow.com/raleigh-nc/luxury-homes/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D

https://www.zillow.com/raleigh-nc/luxury-homes/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D

https://www.zillow.com/raleigh-nc/luxury-homes/3_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22pagination%22%3A%7B%22currentPage%22%3A3%7D%7D

In [23]:
# https://blog.devgenius.io/scraping-zillow-with-python-and-beautifulsoup-bbc7e581c218

#create the first two dataframes
df = pd.DataFrame()
df1 = pd.DataFrame()
#all for loops are pulling the specified variable using beautiful soup and inserting into said variable
for i in soup:
    address = soup.find_all (class_= 'list-card-addr')
    price = list(soup.find_all (class_='list-card-price'))
    beds = list(soup.find_all("ul", class_="list-card-details"))
    details = soup.find_all ('div', {'class': 'list-card-details'})
    home_type = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link = soup.find_all (class_= 'list-card-link')
    
    #create dataframe columns out of variables
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df['links'] = urls
df['links'] = df['links'].astype('str')

#remove html tags
df['links'] = df['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df['links'] = df['links'].replace('" tabindex="0"></a>', ' ', regex=True)

AttributeError: 'NoneType' object has no attribute 'find'